In [1]:
! ls ../../data/

CrowdFlowerAnnotations.txt  Flickr8k.token.txt	       machine_translation
ExpertAnnotations.txt	    Flickr_8k.devImages.txt    readme.txt
Flicker8k_smaller	    Flickr_8k.testImages.txt
Flickr8k.lemma.token.txt    Flickr_8k.trainImages.txt


In [2]:
! pip3 install jieba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=ed27d19a84f32bfb6048db6e791987820e2c52697ae582f47ab1202f2dc61d03
  Stored in directory: /root/.cache/pip/wheels/24/aa/17/5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Layer
import numpy as np
import random
import jieba

batch_size = 16  
epochs = 15  
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
data_path = '../../data/machine_translation/cmn.txt'

input_texts = []
target_texts = []
input_tokens = set()
target_tokens = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# for line in lines[: min(num_samples, len(lines) - 1)]:
for line in random.sample(lines, num_samples):
    input_text, target_text = line.split('\t')
    
    # We use "\t" as the "start sequence" and "\n" as "end sequence"
    target_text = '\t' + target_text + '\n'
    
    tmp = []
    for token in input_text.split(" "):
        token = token.replace(",", "").replace(".", "").replace("?", "").lower()
        tmp.append(token)
        if token not in input_tokens:
            input_tokens.add(token)
    input_texts.append(tmp)
    
    tmp = []
    for token in jieba.cut(target_text, cut_all=False):
        tmp.append(token)
        if token not in target_tokens:
            target_tokens.add(token)
    target_texts.append(tmp)

input_tokens = list(input_tokens)
target_tokens = list(target_tokens)
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

print("\ninput data set:")
print(input_texts[:10])
print("\ntarget data set:")
print(target_texts[:10])

2025-06-16 04:50:17.722328: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-16 04:50:17.722441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 3.182 seconds.
Prefix dict has been built successfully.


Number of samples: 2000
Number of unique input tokens: 2103
Number of unique output tokens: 3140
Max sequence length for inputs: 18
Max sequence length for outputs: 22

input data set:
[['are', 'you', 'a', 'japanese', 'student'], ["let's", 'do', 'it', 'another', 'time'], ['you', "haven't", 'even', 'asked', 'me', 'what', 'i', 'think'], ['they', 'say', 'that', 'he', 'has', 'been', 'dead', 'for', 'two', 'years'], ['people', 'have', 'two', 'legs'], ['do', 'you', 'have', 'other', 'family'], ['lie', 'down', 'and', 'make', 'yourself', 'comfortable'], ["don't", 'give', 'up!'], ['the', 'apple', 'was', 'eaten', 'by', 'me'], ['she', 'was', 'out', 'of', 'japan', 'last', 'year']]

target data set:
[['\t', '你', '是', '日语', '学生', '吗', '？', '\n'], ['\t', '再', '找', '時間', '吧', '。', '\n'], ['\t', '你', '甚', '沒', '問過', '我', '怎麼', '想', '。', '\n'], ['\t', '他们', '说', '他', '死', '了', '两年', '了', '。', '\n'], ['\t', '人有', '两只', '脚', '。', '\n'], ['\t', '你', '有', '其他', '家人', '吗', '？', '\n'], ['\t', '躺', '下', '來', '讓'

In [2]:
input_token_index = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

print("\ninput index:")
print(input_token_index)

print("\ntarget index:")
print(target_token_index)


input index:
{'travel': 0, 'don\'t"': 1, 'model': 2, 'junk': 3, 'water': 4, 'literal': 5, 'washed': 6, 'improved': 7, 'tuesday': 8, 'engaged': 9, "here's": 10, 'feel': 11, 'deposit': 12, 'undone': 13, 'guess': 14, 'yellow': 15, 'superiors': 16, 'poet': 17, 'insisted': 18, 'hoped': 19, 'pretended': 20, 'up': 21, 'genius': 22, 'fine': 23, 'caviar': 24, 'milk': 25, 'severe': 26, 'lock': 27, 'months': 28, 'slowly': 29, 'wonder': 30, "he'll": 31, 'drank': 32, 'single': 33, 'place': 34, 'soul': 35, 'hokkaido': 36, 'subjects': 37, 'homework': 38, 'minutes': 39, 'number': 40, 'cannot': 41, 'computer': 42, 'exam': 43, 'lending': 44, 'opinion': 45, 'spare': 46, 'checking': 47, 'trash': 48, 'ancient': 49, 'house': 50, 'heavier': 51, 'astonish': 52, 'ice': 53, 'both': 54, 'young': 55, 'afraid': 56, 'quickest': 57, 'whose': 58, 'tire': 59, 'away': 60, 'popular': 61, 'stupid': 62, 'cents': 63, 'took': 64, '100': 65, 'wait': 66, 'agree': 67, 'danger': 68, 'robbed': 69, 'take': 70, 'heard': 71, 'upst

In [3]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, token in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[token]] = 1.
    for t, token in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[token]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[token]] = 1.

print("The shape is (batch_size, time_steps, input_dim)")
print("so it will be (number of x, length of each x, dimension of x)")
print("\nencoder input shape:")
print(encoder_input_data.shape)
print("\ndecoder input shape:")
print(decoder_input_data.shape)
print("\ndecoder target shape:")
print(decoder_target_data.shape)
            
print("\nencoder input data (1-hot):")
print(encoder_input_data[-2:])
print("\ndecoder input data (1-hot):")
print(decoder_input_data[-2:])
print("\ndecoder target data (1-hot):")
print(decoder_target_data[-2:])

The shape is (batch_size, time_steps, input_dim)
so it will be (number of x, length of each x, dimension of x)

encoder input shape:
(2000, 18, 2103)

decoder input shape:
(2000, 22, 3140)

decoder target shape:
(2000, 22, 3140)

encoder input data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder input data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder target data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
 

In [4]:
class Encoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = GRU(self.latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')

    def call(self, x):
        output, state = self.lstm(x)
        return output, state


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Encoder(num_encoder_tokens, latent_dim, batch_size)
encoder_outputs, state = encoder(encoder_inputs)

print(f"encoder_inputs: {encoder_inputs.shape}")
print(f"state: {state.shape}")
print(f"encoder_outputs: {encoder_outputs.shape}")

2025-06-16 04:51:06.642431: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-06-16 04:51:06.642967: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-16 04:51:06.643020: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-16 04:51:06.646429: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eb4c86b6f34a): /proc/driver/nvidia/version does not exist
2025-06-16 04:51:06.657325: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

encoder_inputs: (None, None, 2103)
state: (None, 128)
encoder_outputs: (None, None, 128)


In [5]:
import tensorflow as tf

class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        # query is the decoder's current hidden state (h_t)
        # values are the encoder's output states (h_s from s=1 to T_x)

        # query hidden state shape == (batch_size, hidden size)
        # We need to expand dims for broadcasting with values
        query_with_time_axis = tf.expand_dims(query, 1) # (batch_size, 1, hidden size)
        print(f"query_with_time_axis: {query_with_time_axis.shape}")

        # Calculate the score (alignment scores)
        # score shape == (batch_size, max_length, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        print(f"score: {score.shape}")

        # Apply softmax to get attention weights
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        print(f"attention_weights: {attention_weights.shape}")

        # Calculate the context vector
        # context_vector shape == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        print(f"context_vector: {context_vector.shape}")

        return context_vector, attention_weights

In [9]:
class BahdanauAttention2(Layer):
    def __init__(self, units):
        super().__init__()
        
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.attention = tf.keras.layers.AdditiveAttention()

    def call(self, query, value):
        # From Eqn. (4), `W1@ht`.
        w1_query = self.W1(query)

        # From Eqn. (4), `W2@hs`.
        # from memory to key
        w2_key = self.W2(value)

        context_vector, attention_weights = self.attention(
            inputs = [w1_query, value, w2_key],
            # mask=[query_mask, value_mask],
            return_attention_scores = True,
        )

        return context_vector, attention_weights

In [10]:
class Decoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = GRU(latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')
        self.fc = Dense(vocab_size, activation='softmax')
        self.attention = BahdanauAttention2(self.latent_dim)

    def call(self, x, hidden, enc_output):
        print(f"x: {x.shape}")
        print(f"hidden: {hidden.shape}")
        print(f"enc_output: {enc_output.shape}")
        
        rnn_output, rnn_state = self.lstm(x, initial_state=hidden)
        print(f"rnn_output: {rnn_output.shape}")
        
        context_vector, attention_weights = self.attention(rnn_output, enc_output)
        
        # context_and_rnn_output = tf.concat([context_vector, rnn_output], axis=-1)
        
        x = tf.concat([context_vector, rnn_output], axis=-1)
        
        #NOTE: does reshape needed?
        # output = tf.reshape(output, (-1, output.shape[2]))
        y = self.fc(x)

        return y, rnn_state


decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = Decoder(num_decoder_tokens, latent_dim, batch_size)
decoder_outputs, _ = decoder(decoder_inputs, hidden=state, enc_output=encoder_outputs)


print(f"decoder_inputs: {decoder_inputs.shape}")
print(f"state: {state.shape}")
print(f"decoder_outputs: {decoder_outputs.shape}")

x: (None, None, 3140)
hidden: (None, 128)
enc_output: (None, None, 128)
rnn_output: (None, None, 128)
decoder_inputs: (None, None, 3140)
state: (None, 128)
decoder_outputs: (None, None, 3140)


In [11]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 2103)] 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None, 3140)] 0                                            
__________________________________________________________________________________________________
encoder (Encoder)               ((None, None, 128),  857472      input_1[0][0]                    
__________________________________________________________________________________________________
decoder_1 (Decoder)             ((None, None, 3140), 2095812     input_3[0][0]                    
                                                                 encoder[0][0]                

In [12]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
        batch_size=batch_size,
        epochs=3,
        validation_split=0.2)

2025-06-16 04:51:53.908156: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2025-06-16 04:51:54.006039: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2198200000 Hz


Epoch 1/3
x: (16, 22, 3140)
hidden: (16, 128)
enc_output: (16, 18, 128)
rnn_output: (16, 22, 128)
x: (16, 22, 3140)
hidden: (16, 128)
enc_output: (16, 18, 128)
rnn_output: (16, 22, 128)
100/100 [==============================] - ETA: 0s - loss: 2.1807x: (16, 22, 3140)
hidden: (16, 128)
enc_output: (16, 18, 128)
rnn_output: (16, 22, 128)
100/100 [==============================] - 32s 253ms/step - loss: 2.1797 - val_loss: 2.1271
Epoch 2/3
100/100 [==============================] - 22s 222ms/step - loss: 1.9239 - val_loss: 2.2289
Epoch 3/3
100/100 [==============================] - 25s 254ms/step - loss: 1.9166 - val_loss: 2.2670


In [13]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [14]:
encoder_model = Model(encoder_inputs, [encoder_outputs, state])
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 2103)]      0         
_________________________________________________________________
encoder (Encoder)            ((None, None, 128), (None 857472    
Total params: 857,472
Trainable params: 857,472
Non-trainable params: 0
_________________________________________________________________


In [15]:
decoder_state_input = Input(shape=(latent_dim,))
the_encoder_output = Input(shape=(None, latent_dim,))

decoder_outputs, decoder_state = decoder(
    decoder_inputs, hidden=decoder_state_input, enc_output=the_encoder_output)

decoder_model = Model(
    [decoder_inputs] + [decoder_state_input] + [the_encoder_output],
    [decoder_outputs] + [decoder_state])

decoder_model.summary()

x: (None, None, 3140)
hidden: (None, 128)
enc_output: (None, None, 128)
rnn_output: (None, None, 128)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 3140)] 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
decoder_1 (Decoder)             ((None, None, 3140), 2095812     input_3[0][0]           

In [16]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    encoder_output_value, states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, the_decoder_state = decoder_model.predict(
            [target_seq] + [states_value] + [encoder_output_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = the_decoder_state

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

x: (None, 1, 3140)
hidden: (None, 128)
enc_output: (None, 18, 128)
rnn_output: (None, 1, 128)
-
Input sentence: ['are', 'you', 'a', 'japanese', 'student']
Decoded sentence: 我是在。。

-
Input sentence: ["let's", 'do', 'it', 'another', 'time']
Decoded sentence: 我是在。。

-
Input sentence: ['you', "haven't", 'even', 'asked', 'me', 'what', 'i', 'think']
Decoded sentence: 我是在。。

-
Input sentence: ['they', 'say', 'that', 'he', 'has', 'been', 'dead', 'for', 'two', 'years']
Decoded sentence: 我是在。。

-
Input sentence: ['people', 'have', 'two', 'legs']
Decoded sentence: 我是在。。

-
Input sentence: ['do', 'you', 'have', 'other', 'family']
Decoded sentence: 我是在。。

-
Input sentence: ['lie', 'down', 'and', 'make', 'yourself', 'comfortable']
Decoded sentence: 我是在。。

-
Input sentence: ["don't", 'give', 'up!']
Decoded sentence: 我是在。。

-
Input sentence: ['the', 'apple', 'was', 'eaten', 'by', 'me']
Decoded sentence: 我是在。。

-
Input sentence: ['she', 'was', 'out', 'of', 'japan', 'last', 'year']
Decoded sentence: 我是在